In [1]:
import torch
# from options import args
import numpy as np
import pandas as pd
import pickle

In [2]:
device = 'mps'

In [3]:
with open('./data/dataset.pkl','rb') as f:
    dataset = pickle.load(f)

In [4]:
smap = dataset['smap']
reverse_smap = {}
for key in smap:
    reverse_smap[smap[key]] = key

In [5]:
MASK_TOKEN = len(dataset['smap'])+1

In [6]:
# def seq2vec(seq):
# 1. build a dict map movie name to moive id
df = pd.read_csv('Data/ml-20m/movies.csv')


In [7]:
moive2id = {}
id2movie = {}
for i in range(len(df)):
    s = df.loc[i]
    id2movie[s['movieId']] = [s['title'],s['genres']]
    moive2id[s['title']] = s['movieId']

In [8]:
model = torch.load("checkpoint/102.model").to(device)

In [18]:
def pridct(model,movie_seq,smap,reverse_smap,moive2id,id2movie,predict_len):
    seq = []
    for x in movie_seq:
        seq.append(smap[moive2id[x]])
    # then add mask token
    seq.append(MASK_TOKEN)
    seq = seq[-128:]
    padding_len = 128 - len(seq)
    seq = [0] * padding_len + seq
    vec = torch.LongTensor(seq).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        prediction = model(vec)
    masked_predict = prediction[0,-1].cpu().numpy()
    sorted_predicted_ids = np.argsort(masked_predict).tolist()[::-1]
    predicted_id = []
    for x in sorted_predicted_ids:
        if x in seq:
            continue
        if x in reverse_smap:
            predicted_id.append(reverse_smap[x])
        if len(predicted_id)>predict_len:
            break
    res = [id2movie[x] for x in predicted_id]
    for item in res:
        print(item)
    return res

In [19]:
seq = ["Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)",
       'Harry Potter and the Chamber of Secrets (2002)',
       'Harry Potter and the Prisoner of Azkaban (2004)',
       'Harry Potter and the Goblet of Fire (2005)'
      ]
res = pridct(model,seq,smap,reverse_smap,moive2id,id2movie,10)

['Harry Potter and the Order of the Phoenix (2007)', 'Adventure|Drama|Fantasy|IMAX']
['Harry Potter and the Half-Blood Prince (2009)', 'Adventure|Fantasy|Mystery|Romance|IMAX']
['Lord of the Rings: The Two Towers, The (2002)', 'Adventure|Fantasy']
['Chronicles of Narnia: The Lion, the Witch and the Wardrobe, The (2005)', 'Adventure|Children|Fantasy']
['Lord of the Rings: The Fellowship of the Ring, The (2001)', 'Adventure|Fantasy']
['Harry Potter and the Deathly Hallows: Part 2 (2011)', 'Action|Adventure|Drama|Fantasy|Mystery|IMAX']
['Pirates of the Caribbean: The Curse of the Black Pearl (2003)', 'Action|Adventure|Comedy|Fantasy']
['Twilight (2008)', 'Drama|Fantasy|Romance|Thriller']
['Spirited Away (Sen to Chihiro no kamikakushi) (2001)', 'Adventure|Animation|Fantasy']
['Dark Knight, The (2008)', 'Action|Crime|Drama|IMAX']
['Shawshank Redemption, The (1994)', 'Crime|Drama']
